In [1]:
from gensim.models.ldamodel import LdaModel
import gensim.corpora as corpora
import pandas as pd
import gensim

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def word_cut(doc):
    """
    切词
    :param doc:
    :return:2D list
    """
    result = []
    for line in doc:
        result.append(line.split())
    dictionary = corpora.Dictionary(result)
    return result, dictionary

In [11]:
doc = pd.read_csv('../../dataset/020_7_shuffle_train.csv',
                  delimiter='\t',
                  header=None,
                  names=['content', 'label'],
                  encoding='utf8')

doc = doc['content']

rubbish = [r'\[ / LaTeXI ]', r'\[', r'LaTeXI', r'\]', r'\{', r'\}',
           r'& gt ;', r'\\', r'frac', r'matrix', r'\(', r'\)', r'[0-9]']

for rub in rubbish:
    doc.replace(rub, '', inplace=True, regex=True)

doc = doc.values

In [5]:
corpus, dictionary = word_cut(doc)
print(dictionary)

Dictionary(22592 unique tokens: ['-', '/', '^', 'a', 'c']...)


In [6]:
model = gensim.models.Word2Vec(corpus, size=200)

In [31]:
corpus_cbow = [dictionary.doc2bow(text) for text in corpus]

In [34]:
# tfidf = gensim.models.TfidfModel(doc)
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

tfidf=gensim.models.TfidfModel(corpus_cbow)
corpus_tfidf=tfidf[corpus_cbow]

In [32]:
lda = LdaModel(corpus=corpus_cbow, id2word=dictionary, num_topics=1)

In [22]:
# from gensim.test.utils import common_corpus,common_dictionary
# model = LdaModel(common_corpus, id2word=common_dictionary, distributed=True)
# common_corpus

In [61]:
corpus_lda = lda[corpus_tfidf]

In [70]:
lda.get_document_topics(corpus_cbow[11])

[(0, 1.0)]

In [72]:
dictionary.id2token(lda.get_document_topics(corpus_cbow[11]))

TypeError: 'dict' object is not callable

---

In [74]:
import jieba
raw_documents = [
    '无偿检举介绍买卖毒品的行为应如何定性',
    '吸毒男动态持有大量毒品的行为该如何认定',
    '如何区分是非法种植毒品原植物罪还是非法制造毒品罪',
    '为毒贩贩卖毒品提供帮助构成贩卖毒品罪',
    '将自己吸食的毒品原价转让给朋友吸食的行为该如何认定',
    '为获报酬帮人购买毒品的行为该如何认定',
    '毒贩出狱后再次够买毒品途中被抓的行为认定',
    '虚夸毒品功效劝人吸食毒品的行为该如何认定',
    '妻子下落不明丈夫又与他人登记结婚是否为无效婚姻',
    '一方未签字办理的结婚登记是否有效',
    '夫妻双方1990年按农村习俗举办婚礼没有结婚证 一方可否起诉离婚',
    '结婚前对方父母出资购买的住房写我们二人的名字有效吗',
    '身份证被别人冒用无法登记结婚怎么办？',
    '同居后又与他人登记结婚是否构成重婚罪',
    '未办登记只举办结婚仪式可起诉离婚吗',
    '同居多年未办理结婚登记，是否可以向法院起诉要求离婚'
]
corpora_documents = []
for item_text in raw_documents:
    item_str = list(jieba.cut((item_text)))
    corpora_documents.append(item_str)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1.WIN\AppData\Local\Temp\jieba.cache
Loading model cost 1.076 seconds.
Prefix dict has been built succesfully.


In [78]:
dictionary = corpora.Dictionary(corpora_documents)

In [79]:
model = gensim.models.Word2Vec(corpus, size=200)

In [80]:
corpus_cbow = [dictionary.doc2bow(text) for text in corpus]

In [81]:
tfidf=gensim.models.TfidfModel(corpus_cbow)
corpus_tfidf=tfidf[corpus_cbow]

In [92]:
lda = LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=2)

In [93]:
lda.print_topics()

[(0,
  '0.356*"是" + 0.293*"，" + 0.223*"的" + 0.021*"该" + 0.015*"后" + 0.015*"按" + 0.014*"可" + 0.012*"向" + 0.009*"年" + 0.009*"应"'),
 (1,
  '0.373*"为" + 0.173*"与" + 0.104*"，" + 0.081*"的" + 0.068*"将" + 0.041*"可以" + 0.029*"？" + 0.024*"后" + 0.022*"被" + 0.022*"又"')]